In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import copy

################################
# CONST DECLARATIONS
################################
MASK_1 = np.asarray([36, 25, 25])
MASK_2 = np.asarray([86,255,255])

In [2]:
def get_section(img, row1, row2, col1, col2):
    return img[row1:row2, col1:col2]

def get_rows(green):
    rows = []
    last_row = -1
    for i in range(len(green)):
        if last_row == -1:
            if green[i] > 2000:
                last_row = i
        else:
            if green[i] < 2000:
                rows.append((last_row, i))
                last_row = -1
                
    return rows

def reduce_rows(rows, depth):
    new_rows = []
    reduced = False
    for row1 in rows:
        start, end = row1
        for row2 in rows:
            if row2 != row1:
                if np.abs(row2[0] - row1[1]) < 50:
                    end = row2[1]
                    rows.remove(row2)
                    reduced = True
        
        new_rows.append((start, end))
    
    if reduced and depth < 10:
        return reduce_rows(new_rows, depth+1)
    else:
        return new_rows

def get_boxes(rows1, rows2, y1, y2, col_offset):
    boxes = []
    if len(rows1) == len(rows2):
        for i in range(len(rows1)):
            if np.abs(rows1[i][0] - rows2[i][0]) < 150 and np.abs(rows1[i][0] - rows2[i][1]) < 200:
                x1 = min(rows1[i][0], rows2[i][0])+col_offset
                x2 = max(rows1[i][1], rows2[i][1])+col_offset
                boxes.append((x1, y1, x2, y2))
    else:
        rows1, rows2 = fill_rows(rows1, rows2)
        return get_boxes(rows1, rows2, y1, y2, col_offset)
            
    return boxes

def draw_boxes(img, boxes):
    for box in boxes:
        x1, y1, x2, y2 = box
        cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)
        
def fill_rows(rows1, rows2):
    new_rows = []
    
    if len(rows1) > len(rows2):
        short = rows2
        long = rows1
    else:
        short = rows1
        long = rows2

    for i in range(len(long)):
        if all(np.abs(long[i][0]-row[0]) > 150 for row in short): 
            if long[i][0] < 1920//2:
                new_rows.append((long[i][0] - 5, long[i][1] - 5))
            else:
                new_rows.append((long[i][0] + 25, long[i][1] + 25))
        else:
            new_rows.append(short.pop(0))

    return long, new_rows

def get_green_sums(section):
    green_sums = []
    x_range = range(0, len(section[0]))
    y_range = range(0, len(section))
    
    for i in x_range:
        green_sum = 0
        for j in y_range:
            green_sum += section[j][i]
        green_sums.append(green_sum)
        
    return green_sums

def detect_rows(section):
    green_sums = get_green_sums(section)
    return reduce_rows(get_rows(green_sums), 0)

# Mask image and returns
# args: array: image in hsv format
def mask_image(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    masked_img = cv2.inRange(hsv_img, MASK_1, MASK_2)
    return masked_img

In [4]:
#img = cv2.imread("img/test_img.jpg")
#masked_img = mask_image(img)

count = 0

cap = cv2.VideoCapture('vid/Corn_Row_Field_Sun_n_Shade.MP4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('vid/output.MP4', fourcc, 20.0, (1920,1080))
while(cap.isOpened()):
    break
    count+=1
    ret, frame = cap.read()
    if count > 10000:
        masked_img = mask_image(frame)
        for section_start in range(200, 1080, 100):
            section1 = get_section(masked_img, section_start, section_start+50, 240, 1680)
            rows1 = detect_rows(section1)

            section2 = get_section(masked_img, section_start+50, section_start+100, 240, 1680)
            rows2 = detect_rows(section2)

            boxes = get_boxes(rows1, rows2, section_start, section_start+100, 240)
            draw_boxes(frame, boxes)
        
        out.write(frame)
        plt.imshow(frame)
        plt.show()
        if count > 10500:
            break

cap.release()
out.release()

#cv2.imwrite('img/color_lane_detection2.jpg', img)